# What the Func?

What is the need to have Func classes and why not just use functions? What are the additional features that Func classes provide? What are the best
 practices to use Func classes? Here we provide answers to these questions.

## Envirnoment Setup

Here we first create a dataframe of simulated returns which will be used to demonstrate the features of Func classes.

In [2]:
from zpmeta.sources.panelsource import PanelSource
from zpmeta.funcs.func import Func
from zpmeta.singletons.singletons import MultitonMeta
from pandas import DataFrame, Series, concat, MultiIndex, date_range, IndexSlice
import numpy as np
from datetime import datetime
import logging

logging.basicConfig(level=logging.INFO)

In [3]:
class RandomReturns(PanelSource, metaclass=MultitonMeta):
    '''Subclasses Su to create a dataframe of random numbers.
    Accepts a dictionary of parameters, including:
    cols: list of column names
    '''
    def __init__(self, params: dict = None):
        super(RandomReturns, self).__init__(params)
        self.appendable = dict(xs=True, ts=True)
    
    def _execute(self, entities=None, period=None):
        cols = MultiIndex.from_product([val for val in entities.values()], names=entities.keys())
        idx = date_range(period[0], period[1], freq=self.params['freq'])
        result = DataFrame(np.random.randn(len(idx), len(cols)), columns=cols, index=idx)
        
        return result
    

In [4]:
returns_source = RandomReturns(dict(freq='B'))

INFO:root:args: ({'freq': 'B'},) ; kwds: {}
INFO:root:Multiton checking registry for key: (<class '__main__.RandomReturns'>, '{"freq": "B"}')
INFO:root:Multiton No Instance of <class '__main__.RandomReturns'> {"freq": "B"}
INFO:root:Multiton Registering Instance of <class '__main__.RandomReturns'> {"freq": "B"}


Let us print the dataframe

In [5]:
returns_df = returns_source(entities=dict(Type=['A','B','C'], ID=[1,2]), period=(datetime(2019,1,12), datetime(2019,1,31)))
returns_df


INFO:root:RUN RandomReturns {'freq': 'B'}
INFO:root:RUN INITIAL: [{'Type': ['A', 'B', 'C'], 'ID': [1, 2]}] 2019-01-12 00:00:00 - 2019-01-31 00:00:00
INFO:root:EXEC INITIAL: [{'Type': ['A', 'B', 'C'], 'ID': [1, 2]}] 2019-01-12 00:00:00 - 2019-01-31 00:00:00
INFO:root:DONE RandomReturns {'freq': 'B'}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14  0.203427  1.094793  1.238510  0.436179 -0.778542  1.032954
2019-01-15 -2.558625 -1.452874  1.524248 -0.068099  0.459581 -0.225512
2019-01-16  1.220787 -2.148181  1.533597  0.389162 -1.348132 -0.338422
2019-01-17 -2.082212 -0.769242 -0.896823  0.565418  1.537430 -0.838228
2019-01-18  0.768327  0.511928  0.634387 -0.281895  0.170772  0.446515
2019-01-21 -1.237903  0.414620 -1.482883 -0.225134  2.463365  0.496753
2019-01-22  0.002566  0.543653 -0.667197  0.792644 -0.136562 -2.236789
2019-01-23 -0.387090 -0.032432  1.507081 -1.517323  1.568317 -1.593385
2019-01-24  0.536431  1.308444 -0.258392 -0.235853  0.697833 -0.940671
2019-01-25 -0.561824  0.507689 -0.925774  0.196918  1.272855  0.530265
2019-01-28 -0.005969  0.701838 -1.664014 -1.085120 -1.796755  1.154414
2019-01-29  2.396845  0.000656 -2.546944 -0.048270 -1.754068  1.030508
2019-01-30 -0.121565 -0.567974  0.498608 -0.871748  0.079447  1.386067
2019-01-31  0.668481 -0.443019 -1.431296  1.513038  0.991842 -0.078095

## Usage Example

Let us create a Func class that calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.

In [50]:
class RollingVolatility_g_Returns(Func):
    '''Calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.'''
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        return dict(type='lin',lookback = 252)

    @classmethod
    def _execute(cls, operand: DataFrame, params=None) -> DataFrame:
        print(params)
        if params['type'] == 'exp':
            # calculate the rolling exponential volatility
            result = operand.ewm(span=params['lookback']).std()
        elif params['type'] == 'lin':
            # calculate rolling standard  on the operand dataframe
            result = operand.rolling(params['lookback']).std()
        else:
            raise ValueError(f'given type not implemented. type must be either exp or lin')
        
        return result

Now let us use this Func class to calculate rolling volatility of the returns dataframe we created earlier.

In [51]:
vol_func = RollingVolatility_g_Returns(dict(type='lin', lookback=5))

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 5}


This 'vol_func' instance is callable, and it behaves like a usual function. Let us call it on the returns dataframe.

In [52]:
vol = vol_func(returns_df)
vol

Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.086009  0.593954  0.999320  1.218553  0.643075  0.757506
2019-01-21  1.069254  0.933727  1.212101  1.225298  0.771134  0.384689
2019-01-22  1.050456  0.840011  1.147180  1.325645  0.778908  0.691370
2019-01-23  0.321535  0.859208  1.186474  1.300319  1.127243  0.860951
2019-01-24  0.906726  0.736277  0.608601  1.169457  0.880544  0.998377
2019-01-25  1.130853  0.731093  0.824684  1.162417  0.720954  0.958552
2019-01-28  1.145532  0.340105  0.674829  1.214574  1.357433  0.679075
2019-01-29  1.197224  0.351898  0.635490  0.289455  1.382724  0.683939
2019-01-30  1.174764  0.853467  0.952492  0.440490  1.071192  0.675100
2019-01-31  0.829632  0.805830  1.058047  0.443951  1.012452  0.279581

We don't need to create an explicit instance variable to call the Func class. We can call it directly as follows:

In [53]:
vol = RollingVolatility_g_Returns(dict(type='lin', lookback=5))(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 5}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.086009  0.593954  0.999320  1.218553  0.643075  0.757506
2019-01-21  1.069254  0.933727  1.212101  1.225298  0.771134  0.384689
2019-01-22  1.050456  0.840011  1.147180  1.325645  0.778908  0.691370
2019-01-23  0.321535  0.859208  1.186474  1.300319  1.127243  0.860951
2019-01-24  0.906726  0.736277  0.608601  1.169457  0.880544  0.998377
2019-01-25  1.130853  0.731093  0.824684  1.162417  0.720954  0.958552
2019-01-28  1.145532  0.340105  0.674829  1.214574  1.357433  0.679075
2019-01-29  1.197224  0.351898  0.635490  0.289455  1.382724  0.683939
2019-01-30  1.174764  0.853467  0.952492  0.440490  1.071192  0.675100
2019-01-31  0.829632  0.805830  1.058047  0.443951  1.012452  0.279581

## Feature 1: Default Params

Using the _std_params method, we can define default parameters for the Func class. These default parameters are used if the user does not provide 
the params while calling the Func instance. In the above example, we have defined the default params as type='lin' and lookback=252. If the user 
does not provide these params while calling the Func instance, it returns the rolling linear volatility with a lookback of 252 days.

But we can also provide named default params. Let us define a Func class that calculates the rolling volatility as per the RiskMetrics methodology.
 Let us redefine our class as follows:

In [54]:
class RollingVolatility_g_Returns(Func):
    '''Calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.'''
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        if name is None:
            return dict(type='lin',lookback = 252)
        elif name == 'RiskMetrics':
            return dict(type='exp',lookback = 33)
        elif name == 'DRQ':
            return dict(type='lin',lookback = 5)
        else:
            raise ValueError(f'given name not implemented. name must be either None or RiskMetrics')
        
        return dict(type='lin',lookback = 252)

    @classmethod
    def _execute(cls, operand: DataFrame, params=None) -> DataFrame:
        if params['type'] == 'exp':
            # calculate the rolling exponential volatility
            result = operand.ewm(span=params['lookback']).std()
        elif params['type'] == 'lin':
            # calculate rolling standard  on the operand dataframe
            result = operand.rolling(params['lookback']).std()
        else:
            raise ValueError(f'given type not implemented. type must be either exp or lin')
        
        return result

Now let us see the power of default parameters.

In [55]:
vol_func = RollingVolatility_g_Returns(params=('RiskMetrics',{}))
vol = vol_func(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'exp', 'lookback': 33}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15  1.329012  0.600600  1.415864  0.783538  0.646394  1.176705
2019-01-16  1.399919  0.471703  1.001496  0.560694  0.787816  0.979810
2019-01-17  1.207151  0.570722  1.065370  1.331082  0.726449  0.796678
2019-01-18  1.066912  0.604752  0.997308  1.227326  0.624016  0.731394
2019-01-21  1.068157  0.866403  1.226145  1.103668  0.798123  0.647566
2019-01-22  0.988012  0.778708  1.102541  1.256465  0.717732  0.835520
2019-01-23  0.900808  0.718975  1.068268  1.250692  1.030391  0.910875
2019-01-24  1.182256  0.702117  1.044829  1.183581  0.963255  1.111487
2019-01-25  1.164637  0.654028  1.004946  1.119871  0.909380  1.030864
2019-01-28  1.125589  0.631256  0.968288  1.086986  1.106957  0.962350
2019-01-29  1.064889  0.634708  0.911264  1.087226  1.047104  0.914365
2019-01-30  1.009043  0.765385  1.047483  1.025147  1.006193  0.872914
2019-01-31  1.015359  0.723279  1.022016  0.988797  0.950390  0.843139

As we see, just by providing the 'RiskMetrics' label, the Func class now uses the RiskMetrics parameters. Of course, we can do this using regular 
functions, but Func allows us to remember multiple parameters for a given function. This is useful when we have multiple parameters that we want. 
Let us call it again using the 'DRQ' setting.

In [56]:
drq_vol_func = RollingVolatility_g_Returns(params=('DRQ',{}))
vol = drq_vol_func(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 5}


Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.086009  0.593954  0.999320  1.218553  0.643075  0.757506
2019-01-21  1.069254  0.933727  1.212101  1.225298  0.771134  0.384689
2019-01-22  1.050456  0.840011  1.147180  1.325645  0.778908  0.691370
2019-01-23  0.321535  0.859208  1.186474  1.300319  1.127243  0.860951
2019-01-24  0.906726  0.736277  0.608601  1.169457  0.880544  0.998377
2019-01-25  1.130853  0.731093  0.824684  1.162417  0.720954  0.958552
2019-01-28  1.145532  0.340105  0.674829  1.214574  1.357433  0.679075
2019-01-29  1.197224  0.351898  0.635490  0.289455  1.382724  0.683939
2019-01-30  1.174764  0.853467  0.952492  0.440490  1.071192  0.675100
2019-01-31  0.829632  0.805830  1.058047  0.443951  1.012452  0.279581

We can use the named params while overriding some of the elements of the parameters. For example, let us use the RiskMetrics parameters but 
override the lookback to 66 days.

In [57]:
annual_drq_vol_func = RollingVolatility_g_Returns(params=('DRQ',{'lookback': 252}))
vol = annual_drq_vol_func(returns_df)
vol

INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin', 'lookback': 252}


Type         A       B       C    
ID           1   2   1   2   1   2
2019-01-14 NaN NaN NaN NaN NaN NaN
2019-01-15 NaN NaN NaN NaN NaN NaN
2019-01-16 NaN NaN NaN NaN NaN NaN
2019-01-17 NaN NaN NaN NaN NaN NaN
2019-01-18 NaN NaN NaN NaN NaN NaN
2019-01-21 NaN NaN NaN NaN NaN NaN
2019-01-22 NaN NaN NaN NaN NaN NaN
2019-01-23 NaN NaN NaN NaN NaN NaN
2019-01-24 NaN NaN NaN NaN NaN NaN
2019-01-25 NaN NaN NaN NaN NaN NaN
2019-01-28 NaN NaN NaN NaN NaN NaN
2019-01-29 NaN NaN NaN NaN NaN NaN
2019-01-30 NaN NaN NaN NaN NaN NaN
2019-01-31 NaN NaN NaN NaN NaN NaN

As we can see the default DRQ params have been updated with a lookback of 252.

We can also do this overriding of params while calling the Func as a function

In [58]:
vol = annual_drq_vol_func(returns_df, params={'lookback': 11})
vol


Type         A       B       C    
ID           1   2   1   2   1   2
2019-01-14 NaN NaN NaN NaN NaN NaN
2019-01-15 NaN NaN NaN NaN NaN NaN
2019-01-16 NaN NaN NaN NaN NaN NaN
2019-01-17 NaN NaN NaN NaN NaN NaN
2019-01-18 NaN NaN NaN NaN NaN NaN
2019-01-21 NaN NaN NaN NaN NaN NaN
2019-01-22 NaN NaN NaN NaN NaN NaN
2019-01-23 NaN NaN NaN NaN NaN NaN
2019-01-24 NaN NaN NaN NaN NaN NaN
2019-01-25 NaN NaN NaN NaN NaN NaN
2019-01-28 NaN NaN NaN NaN NaN NaN
2019-01-29 NaN NaN NaN NaN NaN NaN
2019-01-30 NaN NaN NaN NaN NaN NaN
2019-01-31 NaN NaN NaN NaN NaN NaN

The difference here is that this override is temporary and does not change the default params of the Func class. 

In [59]:
annual_drq_vol_func.params

{'type': 'lin', 'lookback': 252}

These features are extremely helpful when you have a long list of parameters in quantitative research, with some standardized parameters, but where 
you also need to change some of the parameters for specific use cases.

## Feature 2: Partial Functions

From the above examples, one can see that Func classes can be used to effortlessly create partial functions. Let us look at an example.

In [60]:
class RollingVolatility_g_Returns(Func):
    '''Calculates rolling volatility of returns. It can be used to calculate both linear and exponential rolling volatility.'''
    @classmethod
    def _std_params(cls, name: str = None) -> dict:
        return {}

    @classmethod
    def _execute(cls, operand: DataFrame, params=None) -> DataFrame:
        if params['type'] == 'exp':
            # calculate the rolling exponential volatility
            result = operand.ewm(span=params['lookback']).std()
        elif params['type'] == 'lin':
            # calculate rolling standard  on the operand dataframe
            result = operand.rolling(params['lookback']).std()
        else:
            raise ValueError(f'given type not implemented. type must be either exp or lin')
        
        return result
    
vol_func = RollingVolatility_g_Returns(params={'type': 'lin'})


INFO:root:INIT RollingVolatility_g_Returns {'type': 'lin'}


Now "vol_func" behaves like a partial function. We can feed it any lookback parameter 
that we want.

In [63]:
vol = vol_func(returns_df, {'lookback': 5})
vol

Type               A                   B                   C          
ID                 1         2         1         2         1         2
2019-01-14       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-15       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-16       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-17       NaN       NaN       NaN       NaN       NaN       NaN
2019-01-18  1.086009  0.593954  0.999320  1.218553  0.643075  0.757506
2019-01-21  1.069254  0.933727  1.212101  1.225298  0.771134  0.384689
2019-01-22  1.050456  0.840011  1.147180  1.325645  0.778908  0.691370
2019-01-23  0.321535  0.859208  1.186474  1.300319  1.127243  0.860951
2019-01-24  0.906726  0.736277  0.608601  1.169457  0.880544  0.998377
2019-01-25  1.130853  0.731093  0.824684  1.162417  0.720954  0.958552
2019-01-28  1.145532  0.340105  0.674829  1.214574  1.357433  0.679075
2019-01-29  1.197224  0.351898  0.635490  0.289455  1.382724  0.683939
2019-01-30  1.174764  0.853467  0.952492  0.440490  1.071192  0.675100
2019-01-31  0.829632  0.805830  1.058047  0.443951  1.012452  0.279581